<a href="https://colab.research.google.com/github/Iamsdt/UdacityPyTorch/blob/master/src/notebook/PretrainedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms, models

import helper

In [0]:
# create train and test loader
data_dir = 'Cat_Dog_data/Cat_Dog_data'

# Define transforms for the training data and testing data
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [0]:
# load a pretrained model
model = models.densenet121(pretrained = True)

In [0]:
# model current classifer
print(model.classifier)

Linear(in_features=1024, out_features=1000, bias=True)


In [0]:
# Freeze the parameters
for param in model.parameters():
  param.required_grad = False

In [0]:
from collections import OrderedDict
from torch import nn
classifier = nn.Sequential(OrderedDict([
                ('fc1', nn.Linear(1024, 500)),
                ('relu', nn.ReLU()),
                ('fc2', nn.Linear(500, 2)),
                ('output', nn.LogSoftmax(dim=1))
                ]))

# replace the classifer
model.classifier = classifier

In [0]:
# check the classifier
print(model.classifier)

In [0]:
from torch import optim
# move model to cuda (GPU)
# model.to("cuda")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
model.to(device)

In [0]:
# train model
epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

Epoch 1/1.. Train loss: 1.031.. Test loss: 0.217.. Test accuracy: 0.937
Epoch 1/1.. Train loss: 0.449.. Test loss: 0.269.. Test accuracy: 0.877
Epoch 1/1.. Train loss: 0.352.. Test loss: 0.155.. Test accuracy: 0.942
Epoch 1/1.. Train loss: 0.235.. Test loss: 0.108.. Test accuracy: 0.959
Epoch 1/1.. Train loss: 0.172.. Test loss: 0.080.. Test accuracy: 0.973
Epoch 1/1.. Train loss: 0.174.. Test loss: 0.065.. Test accuracy: 0.975
Epoch 1/1.. Train loss: 0.173.. Test loss: 0.062.. Test accuracy: 0.978
Epoch 1/1.. Train loss: 0.165.. Test loss: 0.056.. Test accuracy: 0.980
Epoch 1/1.. Train loss: 0.190.. Test loss: 0.056.. Test accuracy: 0.979
Epoch 1/1.. Train loss: 0.140.. Test loss: 0.056.. Test accuracy: 0.979
Epoch 1/1.. Train loss: 0.148.. Test loss: 0.052.. Test accuracy: 0.979
Epoch 1/1.. Train loss: 0.153.. Test loss: 0.052.. Test accuracy: 0.978
Epoch 1/1.. Train loss: 0.180.. Test loss: 0.084.. Test accuracy: 0.971
Epoch 1/1.. Train loss: 0.246.. Test loss: 0.117.. Test accuracy